In [2]:
# Read the data into pandas-dataframe

import pandas as pd
df1= pd.read_csv('./Seattle_house_prices_test.csv')
df2= pd.read_csv('./boston_house_prices.csv')


In [3]:
df1.head()

,beds,baths,size,size_units,lot_size,lot_size_units,zip_code,price,MEDV
0,3,3.0,2850,sqft,4200.00,sqft,98119,1175000,24.0
1,4,5.0,3040,sqft,5002.00,sqft,98106,1057500,21.6
2,3,1.0,1290,sqft,6048.00,sqft,98125,799000,54.4
3,3,2.0,2360,sqft,0.28,acre,98188,565000,33.4
4,3,3.5,1942,sqft,1603.00,sqft,98107,1187000,36.2


In [4]:
df2.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,price
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,1175000.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,1057500.0
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,799000.0
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,565000.0
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2,1187000.0


In [5]:
'''
    Cartesian product function: takes 2 dataframes(df1, df2) as input, 
    returns the output(newdf) as the cartesian product of the input tables
'''
def cartJoin(df1, df2):
    cols=list(df1.columns)+ list(df2.columns)
    indx= df1.shape[0]+ df2.shape[0]

    # print("cols= ", cols)

    newdf= pd.DataFrame(columns= cols, index= range(0,indx) )

    k=0
    for row1 in df1.itertuples( ):
        tmp = list(row1) 
        # tmp = list(row1) 
        for row2 in df2.itertuples():
            row1= tmp[1:] 
            row2=list(row2[1:])
            row1.extend(row2)
            newdf.loc[k]= row1 
            k+=1

    # for row1 in df1.itertuples( ):
    #     for row2 in df2.itertuples():
    #         newRow.append(newRow ,row1+ row2)

    # print("newdf:\n****************\n", newdf)
    # newdf.to_csv(outputfile)
    return newdf

In [6]:
'''
    Natural Join function(natJoin): takes 2 dataframes(df1, df2) as input, 
    returns the output(newdf) as the natural join of the input tables
'''

def natJoin(df1, df2):
    indx= df1.shape[0]+ df2.shape[0]

    # print(cols)
    # print('len of cols: ',len(cols))
    comm_attr= set(df1.columns) & set(df2.columns)
    # print(comm_attr)

    cols=[]
    for attr in df1.columns:
        cols.append(attr)
    for attr in df2.columns:
        if attr not in comm_attr:
            cols.append(attr)

    # new_col= new_col + list(comm_attr)
    # print("cols= ", cols)
    # print('len of cols: ', len(cols))

    #extract indices of common attributes as tuples
    comm_row_idx=[]
    for i in range(len(df1.columns)):
        for j in range(len(df2.columns)):
            if df1.columns[i] == df2.columns[j]:
                comm_row_idx.append((i,j))   ##### a list of tuples- (i,j) where i= index of a common attr in df1, j= index of a common attr in df2,

    # print('comm_row_idx: ',comm_row_idx)

    newdf= pd.DataFrame(columns= cols, index= range(0,indx))

    k=0
    for row1 in df1.itertuples( ):
        tmp = list(row1) 
        # separate the row1 attr which are in comm_attr;
        # when they appear in row2, compare their values and add to the df only if values are equal. 
        # comm_val= tmp[1:][i]   
        for row2 in df2.itertuples():
            row1= tmp[1:]
            row2=list(row2[1:])
            good_add=[]
            for i, j in comm_row_idx:
                # print("i,j", i,j)
                if row2[j]== row1[i]:
                    #remove common attr values from row2 
                    row2.remove(row2[j])
                    good_add.append(True)
                else:
                    good_add.append(False)
            if sum(good_add)== len(comm_row_idx):
                row1.extend(row2)
                # print("row to be put in newdf: ", row1)
                newdf.loc[k]= row1 
                k+=1


    # for row1 in df1.itertuples( ):
    #     for row2 in df2.itertuples():
    #         newRow.append(newRow ,row1+ row2)

    # print("newdf:\n****************\n", newdf)
    # newdf.to_csv('outputfile.csv')
    return newdf



In [7]:
# WHERE student.class = “I MTech”
# WHERE account.balance >= 100000
'''
    Function to return the result of the where clause
'''
def where_clause(df, qline):
    if not qline.startswith('WHERE'):
        return None
    ################ This is a map of key= symbols, values= lambda functions ##############
    operator_mapping = {
    # x,y,z = df, attr, value
    '<': (lambda x,y,z : x[x[y]< z]),
    '<=':(lambda x,y,z : x[x[y]<= z]),
    '==':(lambda x,y,z : x[x[y]== z]),
    '!=':(lambda x,y,z : x[x[y]!= z]),
    '>=':(lambda x,y,z : x[x[y]>= z]),
    '>': (lambda x,y,z : x[x[y]< z]),
    }
        
    
    #preprocess the query before passing it to df
    # qline= qline[6:].replace("table1", "df")

    qline= qline[6:].split(' ')
    # qline[0]= "df.beds" changes to "beds"
    # dotIdx= qline[0].index('.')
    # qline[0]= qline[0][dotIdx+1:]
    

    qline[0]= qline[0][qline[0].index('.')+1 :]
    
    # print("qline:",qline)
    
    # print("x:", x)
    # print("operator:", operator)
    # print("y:", y)
    
    try:
        value= float(qline[2])
    except: 
        value= qline[2]
        
    
    return operator_mapping[qline[1]](df, qline[0] , value) 
    
    
    # return df.loc[df.beds==3]
    
    # return df[operator_mapping[qline[1]]]

### Query to check the functionality of the code above
# tmp= where_clause(df1, "WHERE jhjkfh.size_units == sqft")

tmp= where_clause(df1, "WHERE kjlk.lot_size_units != sqft")
tmp



    

,beds,baths,size,size_units,lot_size,lot_size_units,zip_code,price,MEDV
3,3,2.0,2360,sqft,0.28,acre,98188,565000,33.4
6,1,1.0,756,sqft,NaN,NaN,98105,480000,22.9


In [8]:
''' Order by clause implementation        WITHOUT      using pandas in-built functions'''
# The ORDER BY clause should support sorting on an attribute in ascending or descending order.
# If no ordering is specified, the default is ascending order

# ORDER BY student.reg_no ASC
# ORDER BY account.balance DESC

def getSecond(val):
    return val[1]

def orderby_clause(df, qline): 
    '''######################## ASSUMPTION- only one attribute to sort the table with ################'''
    qline = qline[9:].split()
    # print(qline)
    
    attr= qline[0][qline[0].index('.')+1 :]
    # print(attr)
    
    # print(qline[1])
    
    tuples= [(i, df.loc[i, attr]) for i in df.index]
    # print(tuples)
    ######################## Sorting WITHOUT  using DataFrame.sort_values()#########################
    if len(qline)> 1 and qline[1]=='DESC':
        tuples.sort(key=getSecond, reverse= True)
    else:
        tuples.sort(key= getSecond)
    
    idxlist= [ tuples[i][0] for i in range(len(tuples))]
    
    # print(df)
    return df.reindex(idxlist)
    
    
# df= orderby_clause(df1, "ORDER BY df1.beds DESC")
# df
    

In [10]:
'''
    Take input from the user 
'''

s='1'
queries= []

################ this loop takes input from user until user enters an empty string, ie until two consecutive \n characters are input #####
while s!= "":
    s= input("Enter the queries: \n")
    queries.append(s)

print("Your queries are: ", str(queries))

Enter the queries: 
 


Your queries are:  ['']


In [11]:
# Example 1
# SELECT *
# FROM student.csv

# Example 2
# SELECT *
# FROM student.csv
# WHERE student.class = “I MTech”
# ORDER BY student.reg_no ASC;

# Example 3
# SELECT customer.name, account.balance
# FROM customer.csv NATURAL JOIN account.csv
# WHERE account.balance >= 100000
# ORDER BY account.balance DESC;



In [13]:
''' 
    Following is an example query; the user is supposed to/ requested to enter all queries in this format
'''

# queries= "SELECT seatle.beds,boston.AGE\n FROM seatle.csv NATURAL JOIN boston.csv \nWHERE seatle.beds >= 3\nORDER BY boston.price DESC"
queries= "SELECT seatle.beds, seattle.size\n FROM seatle.csv\n ORDER BY seatle.size"
queries= queries.split('\n')
# print(queries)
orderbyquery=[]
fromquery=[]
wherequery=[]
orderbyquery=[]
limitquery=[]

deleteIsPresent=False
whereIsPresent=False
orderIsPresent=False
limitIsPresent=False

selectquery= queries[0]

try:
    fromquery= queries[1]
except:
    print("No FROM query was present, Invalid Query")


try:
    wherequery= queries[2]
    whereIsPresent= True
except:
    None

try:
    orderbyquery= queries[3]
    orderIsPresent= True
except:
    None


print(selectquery, fromquery, wherequery, orderbyquery)
# df1= pd.read_csv('./Seattle_house_prices_test.csv')
# df2= pd.read_csv('./boston_house_prices.csv')


SELECT seatle.beds, seattle.size  FROM seatle.csv  ORDER BY seatle.size []


In [14]:
# FROM student.csv
# FROM customer.csv NATURAL JOIN account.csv

# n= len(queries)
'''################## process the FROM clause  ##################################'''

df1= pd.read_csv('./Seattle_house_prices_test.csv') 
df2= pd.read_csv('./boston_house_prices.csv')

if "NATURAL" in fromquery:
    product= natJoin(df1, df2)
    # product = pd.merge(df1, df2) #, how= 'cross') #, on='key') #when on=None, then join is natural, ie common attributes are considered
    # product.to_csv(outputfile)
elif ',' in fromquery:
    product= cartJoin(df1, df2)
else :
    product= df1
    # product= pd.merge(df1, df2, how='cross')
# print("DF at the end of FROM query: ", product.head())
product.to_csv('./outputs/afterFromClause.csv')

''' #########################  process the where clause  ###############################'''
if whereIsPresent:
    #tmp= where_clause(df1, "WHERE kjlk.lot_size_units != sqft")
    product= where_clause(product, wherequery)
    product.to_csv('./outputs/afterWhereClause.csv')

''' ######################## process the orderby clause  ##############################'''
if orderIsPresent:
    # df= orderby_clause(df1, "ORDER BY df1.beds DESC")
    product= orderby_clause(product, orderbyquery)
    product.to_csv('./outputs/afterOrderbyClause.csv')    

''' ######################## process the select clause  ##############################'''
if "*" in selectquery:
    product.to_csv('./outputfile.csv')   
    
else:
    cols= selectquery[7:].split(',')
    cols= [attr.strip() for attr in cols]
    for i in range(len(cols)):
        cols[i] =cols[i][cols[i].index('.')+1 :]
    # print(cols)
    # print(product[cols])
    product[cols].to_csv('./outputfile.csv')


AttributeError: 'NoneType' object has no attribute 'to_csv'